<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

In [2]:
# avoid decoding problems
df = pd.read_csv("train.csv")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [86]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = spacy.load('en_core_web_sm')

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(df['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m'] = list(vecs1)


  0%|                                                                           | 669/404290 [00:07<1:08:22, 98.39it/s]


KeyboardInterrupt: 

In [9]:
vecs2 = []
for qu2 in tqdm(list(df['question2'])):
    doc2 = nlp(qu2) 
    mean_vec2 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m'] = list(vecs2)

100%|████████████████████████████████████████████████████████████████████████| 404290/404290 [1:15:53<00:00, 88.79it/s]


In [96]:
df

,id,qid1,qid2,question1,question2,is_duplicate,q1_feats_m,q2_feats_m
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[211.12986397743225, -144.68305885791779, -68....","[151.2685260772705, -127.01316797733307, -31.5..."
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[144.124685049057, -114.0124841555953, -111.71...","[152.02309501171112, -44.95538953691721, -103...."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[81.75789752602577, -142.18450701236725, 0.559...","[4.930219948291779, -29.029581308364868, -117...."
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[-126.6519216299057, -59.74716019630432, -67.7...","[-6.951928615570068, -44.951730847358704, -17...."
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[299.4440441727638, -188.63200092315674, -22.9...","[96.1745240688324, -71.61394834518433, 21.5848..."
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,"[105.98917961120605, -202.81071376800537, -1.7...","[85.69822990894318, -163.7301703095436, -128.9..."
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,"[-50.411580085754395, -23.510990619659424, -69...","[115.0804762840271, -79.49389886856079, -8.057..."
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,"[-1.5781493782997131, -25.470666706562042, -41...","[10.792293675243855, -45.666466146707535, -51...."
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,"[-2.9045279026031494, 10.836925834417343, -40....","[-18.634379386901855, -0.04308807849884033, -5..."
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,"[212.45816817879677, -168.03972655534744, -54....","[104.10098648071289, -153.590188190341, -59.53..."


In [10]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [11]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [12]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,93,93,100,0.982759
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,5.0,12.5,86,63,66,75,0.596154
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,4.0,12.0,66,66,54,54,0.166667
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,12.0,36,36,35,40,0.039216
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,67,47,46,56,0.175000


In [13]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [14]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,211.129864,-144.683059,-68.811247,-153.662141,-89.931593,2.311301,136.743747,50.449112,-64.150964,56.627526,...,33.176593,-102.487002,144.595747,50.614258,-21.385940,36.385601,-70.999820,4.141466,-131.379525,71.884573
1,144.124685,-114.012484,-111.716694,-104.885038,-88.238478,16.441834,58.238013,102.095138,6.026966,178.498497,...,67.386291,22.131810,40.210794,0.254035,98.600184,-36.436027,-123.609728,71.788337,-15.374614,-10.503958
2,81.757898,-142.184507,0.559867,-104.660084,-84.156631,22.515110,115.521661,50.436953,-111.740923,51.713310,...,61.731855,-49.822495,107.080594,-11.431423,-40.667739,-26.771786,-12.482071,1.764694,-100.330459,148.061206
3,-126.651922,-59.747160,-67.763201,-138.114731,-101.038699,88.148523,-22.912261,85.941426,27.784233,50.810650,...,117.847457,-13.044505,-1.643167,78.158408,23.748127,12.945961,-8.807293,123.360096,64.057049,-24.943289
4,299.444044,-188.632001,-22.946291,-273.683355,-188.480395,107.123044,174.946302,-72.042341,-98.290527,137.439973,...,62.256713,-173.503679,249.320174,27.972507,-26.665078,-28.469215,-62.335434,2.738905,-120.762699,111.779907


In [15]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,151.268526,-127.013168,-31.546286,-142.905807,-97.249094,9.485758,106.682259,36.754201,-36.541905,53.162199,...,28.763393,-108.306631,131.115633,42.530936,-14.726235,15.763654,-59.729557,-9.563384,-99.812549,57.643628
1,152.023095,-44.955390,-103.559249,-128.467601,-118.567610,44.577916,137.906144,26.984746,-78.328355,86.576880,...,117.588061,5.239833,56.622487,92.054093,141.063301,-30.466199,-25.418970,87.120086,-23.594506,7.936921
2,4.930220,-29.029581,-117.808812,-98.332275,-19.064096,-9.867805,141.808202,91.269564,50.727205,12.816846,...,128.699958,17.433420,58.203339,88.642941,-84.962783,71.462342,-10.853897,-59.501685,-99.425331,95.334773
3,-6.951929,-44.951731,-17.343082,-61.444452,-7.469152,16.942014,95.049250,-2.631600,-13.050916,-28.038393,...,70.475260,-31.174277,86.080290,49.967751,25.750448,-27.612377,24.050802,-3.504152,-46.115810,56.242265
4,96.174524,-71.613948,21.584882,-92.742468,-106.643129,10.646790,92.190157,-40.565982,-34.739525,56.340519,...,18.852815,-70.416546,87.693604,38.386024,-14.003138,-12.975101,46.318846,-25.159640,-53.098664,-2.351894


In [16]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [17]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv')

In [98]:
df3_q1['id']=df1['id']
df3_q2['id']=df1['id']
df1  = df1.merge(df2, on='id',how='left')
df2  = df3_q1.merge(df3_q2, on='id',how='left')
result  = df1.merge(df2, on='id',how='left')

In [100]:
result

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,...,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,...,28.763393,-108.306631,131.115633,42.530936,-14.726235,15.763654,-59.729557,-9.563384,-99.812549,57.643628
1,1,0,0.799984,0.399996,0.749981,0.599988,0.699993,0.466664,0.0,1.0,...,117.588061,5.239833,56.622487,92.054093,141.063301,-30.466199,-25.418970,87.120086,-23.594506,7.936921
2,2,0,0.399992,0.333328,0.399992,0.249997,0.399996,0.285712,0.0,1.0,...,128.699958,17.433420,58.203339,88.642941,-84.962783,71.462342,-10.853897,-59.501685,-99.425331,95.334773
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,70.475260,-31.174277,86.080290,49.967751,25.750448,-27.612377,24.050802,-3.504152,-46.115810,56.242265
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,...,18.852815,-70.416546,87.693604,38.386024,-14.003138,-12.975101,46.318846,-25.159640,-53.098664,-2.351894
5,5,1,0.666656,0.571420,0.888879,0.799992,0.705878,0.705878,1.0,0.0,...,140.383325,31.584976,-52.553733,-17.341836,174.319444,27.591935,-57.281026,-11.223623,-83.622666,83.007612
6,6,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,56.540514,-108.176404,112.680577,-9.053159,43.782012,31.403739,-124.683066,71.874762,-26.382156,-52.167546
7,7,1,0.499975,0.499975,0.599988,0.428565,0.571420,0.444440,1.0,0.0,...,22.470603,11.727317,94.625395,-4.702483,-25.395090,35.628580,26.318907,-69.046787,-21.267080,20.628103
8,8,0,0.999950,0.499988,0.999975,0.799984,0.857131,0.749991,0.0,1.0,...,79.278840,-39.944202,-5.888271,60.124768,30.463961,44.714086,31.574358,18.763351,2.371304,1.118337
9,9,0,0.599988,0.499992,0.333322,0.249994,0.444440,0.444440,0.0,0.0,...,-6.159834,-61.199011,-8.392370,59.016275,43.735897,17.042343,-20.888084,-30.897401,30.675718,12.386427
